# **과제2 : 챗봇 만들기2**

## 0.미션

* 예비 에이블러들을 위한 QA 챗봇 모델 만들기2
    * Vector DB에 데이터 추가하기
    * Retriever, memory, LLM를 연결하기
    * 실행시 이력 DB 생성하고 기록하기
    * test

## **1.환경준비**

### (1) 라이브러리 Import

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime

import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [2]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key) 

sk-proj-V9udv6pPbDp09TLcHfAdT3BlbkFJj0QKev8PcCOQI4zaa4Wg


* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [3]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키' 
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB 만들기**

* 데이터 로딩 
    * 1일차에서 제공한 csv 파일의 구조를 그대로 이용
    * 에이블스쿨 홈페이지 FAQ 데이터 수집(https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056)
        * 모든 질문을 csv 형태로 저장
    * 데이터프레임으로 저장하기

* 벡터 데이터베이스
    * 1일차 벡터 데이터베이스를 그대로 이용
        * Embedding 모델 : text-embedding-ada-002
        * DB 경로 : ./db



* 벡터 데이터베이스

In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

database = Chroma(persist_directory="database",  # 경로 지정(현 위치에서 db 폴더 생성)
                    embedding_function = embeddings  # 임베딩 벡터로 만들 모델 지정
)

* 데이터 입력
    * 기존 입력을 모두 제거하고 추가 사항만 모두 입력
    * meta data로 '구분' 칼럼 값 추가하기

* db_aivle.db 내에 데이터 입력하기

In [5]:
input_list = ['최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.',
                            '35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교육입니다. 단, 모집시점에 35세라도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다. (예: 5기 지원 기준 1989년 1월 1일 이후 출생자)',
                            '미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함이 있는 근로인 경우,\n2) 고용보험에 미가입한 경우,\n3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우\n단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.',
                            '직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가능합니다. 향후 취업여부를 확인할 예정입니다.',
                            '아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 15시간/주 미만 근로인 경우에만 미취업자로 간주하여 지원이 가능합니다.\n고용보험에 가입되어 있지 않는 경우에는 특수형태근로자/고용보험미가입근로자로 분류되어 근로시간과 훈련시간이 중복되지 않는다면 훈련 수강은 가능하나, 훈련장려금이 지급되지 않습니다.\n다만, 특수형태근로자의 경우 해촉증명서, 퇴사 사실확인증명원(사업주, 근로자 직인/서명포함)을 제출하는 경우 훈련장려금 지급이 가능하므로,\n고용형태 관련 세부사항은 거주지 관할 고용센터(HRD-Net 확인) 또는 훈련 권역별 고용센터로 문의하시기 바랍니다.\n※ 수도권: 성남고용센터\n충남/충북: 대전고용센터\n대구/경북: 대구고용센터\n전남/전북: 광주고용센터\n부산/경남: 부산고용센터\n- 고용노동부 대표번호 1350, 거주지 근처 지도검색, HRD-Net 메인 하단 지역별 기관검색에서 확인 가능',
                            '사업자도 지원할 수 있나요?\n사업자는 자영업자로 분류됩니다. 자영업자의 경우 관할 고용센터에서 국민내일배움카드 발급 여부를 검토하며, 카드가 발급되어 교육을 수강하여도 훈련장려금은 지급되지 않습니다. 다만, 고용보험을 임의가입한 자영업자의 경우 훈련장려금이 지급됩니다.',
                            '프리랜서도 지원할 수 있나요?\n고용보험에 가입하지 않은 경우 미취업자로 보지만, 프리랜서 계약의 경우 개인별 상황이 다를 수 있어서 관할 고용센터에 문의하여 확인하시기 바랍니다.\n※ 관할 고용센터 :\n수도권: 성남고용센터\n충남/충북: 대전고용센터\n대구/경북: 대구고용센터\n전남/전북: 광주고용센터\n부산/경남: 부산고용센터\n- 고용노동부 대표번호 1350, 거주지 근처 지도검색, HRD-Net 메인 하단 지역별 기관검색에서 확인 가능',
                            '현재 군 복무중인 상태인데, 지원이 가능한가요?\n현재 군 복무중인 상태여도 지원 가능합니다. 다만, 교육 시작 전일까지 병역의무를 마쳐야 교육 수강이 가능합니다.',
                            '퇴사 후 실업급여 수령 중인데, 지원할 수 있나요?\n퇴사 후 고용보험 상실이 완료되어 실업급여를 수급 중인 경우, 훈련 수강에는 문제가 없습니다. 다만, 실업급여와 훈련장려금은 중복 수급이 불가하여 실업급여 지급이 종료된 후 훈련장려금 지급이 개시됩니다.',
                            '다른 K-DT 과정을 들었는데 지원할 수 있나요?\nK-DT 교육과정은 최초 1회에 한해서만 무료 수강이 가능하며, KT 에이블스쿨은 본인 부담금이 발생하는 경우 지원이 어렵습니다. K-DT 과정마다 차이가 있을 수 있으니 개인별 정확한 수강 가능 여부는 관할 고용센터에 확인 부탁드립니다. 관할 고용센터 정보는 고용노동부 대표번호 1350 또는 거주지 근처 지도검색, HRD-Net에서 확인 가능합니다.',
                            'K-DT가 아닌 국비과정 참여 중인데 지원할 수 있나요?\n기존에 수강하신 과정이 K-DT 과정이 아니라면 지원이 가능합니다. 개인별 상황이 다를 수 있으므로 정확한 수강 가능 여부는 관할 고용센터에 문의하여 확인하시기 바랍니다. 관할 고용센터 정보는 고용노동부 대표번호 1350 또는 거주지 근처 지도검색, HRD-Net에서 확인 가능합니다.',
                            '이전에 불합격했을 경우 이번에 다시 지원할수 있나요?\n지원자격에 해당되는 경우 다시 지원이 가능합니다.',
                            '합격 후 과정을 포기하면 재지원이 불가능한가요?\n교육 등록신청을 하지 않거나 교육 일정 시작 전에 교육 등록을 취소해도 지원자격에 해당하는 경우 다시 지원이 가능합니다. 다만, 교육 과정 확정자신고일(교육시작일로부터 7일)이 지난 후 중도 퇴교를 할 경우 과정을 수강한 것으로 간주되며, 향후 K-Digital Training (K-DT) 과정에 재지원은 가능하나 무료 수강은 어렵습니다.',
                            '추가합격제도가 있나요?\n현재 KT 에이블스쿨은 추가합격 제도를 운영하고 있지 않습니다. 향후 추가합격을 운영하는 경우, 사전에 공지할 예정입니다.',
                            '면접은 어떻게 진행되나요?\nKT 에이블스쿨은 별도의 면접 과정이 없습니다 ',
                            '코딩테스트는 어떻게 진행되나요?\n코딩테스트는 AI개발자 Track 지원자를 대상으로 진행되며, 사용 가능한 프로그래밍 언어는 C, C++, JAVA, Python, JavaScript 5가지입니다. 자세한 내용은 서류전형 합격자 대상으로 안내드릴 예정입니다.',
                            '국민내일배움카드가 있어야 인적성검사 응시할 수 있나요?\n아닙니다. 국민내일배움카드는 교육시작일 기준 D-7(주말∙공휴일 제외)까지 발급해 주시고, 최종합격 이후 합격자 대상으로 일괄 안내 드릴 예정입니다.',
                            '인적성검사는 KT 채용과 같은 과목인가요?\n현재 KT 에이블스쿨은 KT 채용 인적성검사와 같은 과목의 인적성검사를 운영하고 있습니다.',
                            '국민내일배움카드가 있어야만 지원할 수 있나요?\n네, 국민내일배움카드 보유자 또는 발급 가능자여야 지원이 가능합니다. 현재 카드를 보유하고 있지 않더라도, K-Digital Training 교육 대상자로 확인되어 신규 발급하면 지원 가능합니다.',
                            '(지원서 작성) 현재 4학년 1학기 재학 또는 휴학 등의 사유로 졸업(예정)증명서 발급이 안되는 경우 어떻게 해야 하나요?\n졸업예정증명서 발급이 불가능한 경우 수료증명서 또는 재학증명서를 제출하여 주시기 바라며, 추후 졸업예정증명서 발급 가능 시점에 제출 요청을 드릴 수 있습니다.',
                            '(지원서 작성) 대학원 재학 중으로 성적표가 없는 경우 어떻게 하나요?\n대학원 재학 중으로 성적증명서가 없으신 경우, 최종학력은 대학교로 입력하여 주시고 졸업(예정)증명서 및 성적증명서 제출란에는 재학증명서를 제출하여 주시기 바랍니다.',
                            '(지원서 작성) 미국 4년제 대학의 경우 증빙서류가 무엇인가요?\n졸업을 확인할 수 있는 서류인 공식 졸업장(예: Diploma)과, 4년간의 이수 과목과 성적을 확인할 수 있는 성적표(예: Transcript)를 제출하시면 됩니다.',
                            '컴퓨터공학 비전공자도 커리큘럼을 따라갈 수 있을까요?\n비전공자도 SW개발에 관심과 열정이 있으신 분들은 충분히 따라 갈 수 있습니다. 이 경우 데이터 사이언스 기반으로 산업 현장에 AI를 접목하고 DX를 이끄는 DX컨설턴트 Track을 추천 드립니다. SW기초 지식 및 코딩역량을 보유하고 있고 개발자로 성장하고 싶은 분께는 AI개발자 Track을 추천 드립니다.',
                            '교육 전에 미리 공부하면 좋은 것이 있을까요?\n파이썬 프로그래밍 사용법과 AI 개념에 대한 기초 이해를 공부하고 오시면 도움이 되실 것 같습니다.',
                            '출결 기준이 어떻게 될까요?\n당일 소정훈련시간의 50퍼센트 미만을 수강한 경우, 2. 지각, 조퇴 3회 누적 시, 결석 처리 됩니다.',
                            '휴가를 사용할 수 있나요?\nK-Digital Training (K-DT) 규정상 월 1회 휴가 사용이 가능합니다.',
                            '교육 수료 기준은 무엇인가요?\nKT 에이블스쿨은 고용노동부 K-Digital Training 사업으로 운영됨에 따라 총 훈련일수의 80% 이상을 출석한 경우 수료대상이 됩니다.\n※ 국민내일배움카드 운영규정 상 단위기간 50%이상 무단결석 시 또는 총 결석 일수가 전체 훈련일수의 20% 초과 시 제적대상이나, 훈련 수준 유지를 위하여 KT 에이블스쿨에서는 100% 출석을 권장하고 있습니다. 출결 미달로 인해 훈련 수준 유지가 어려운 경우 제적 대상이 될 수 있습니다.(#국민내일배움카드 운영규정 제35조)',
                            '교육 장소(지역)는 어떻게 되나요?\nKT 에이블스쿨은 전국 어디서나 참여할 수 있도록, 온/오프라인을 병행하여 탄력적으로 운영합니다.\n오프라인 교육장소는 6개 권역(수도권, 강원권, 부산/경남권, 대구/경북권, 충남/충북권, 전남/전북권)에 위치하고 있습니다.\n수도권 : KT분당교육장 (경기도 성남시 분당구 불정로 90), KT전농교육장 (서울시 동대문구 사가정로 105)\n강원권 : KT원주연수원 (강원도 원주시 행구동 행구덕현길 109)\n부산/경남권 : 에이블스쿨 BDIA 동구캠퍼스 (부산 동구 초량중로29 KCA빌딩)\n대구/경북권 : KT북대구빌딩 (대구광역시 북구 고성로 141)\n충남/충북권 : KT탄방빌딩 (대전광역시 서구 문정로 48번길 30)\n전남/전북권 : KT신안빌딩 (광주광역시 북구 신안동 무등로 202번길)',
                            '지원 지역(교육 장소)을 변경할 수 있나요?\n서류 접수 마감 이후에 지원 지역(교육 장소) 변경은 불가능하며, 최종 합격된 지역에서만 교육이 가능합니다.',
                            '대면 교육도 진행하나요?\n교육 과정 중 STEP1과 STEP2는 대부분 비대면 교육으로 진행하며, 일부 프로젝트는 대면으로 진행될 수 있습니다. STEP3 프로젝트 수행기간에는 대면 & 비대면을 병행하여 진행합니다. (단, 코로나 정부방역지침에 따라 변경될 수 있습니다.)',
                            '비대면으로 진행된다면 개인이 원하는 시간대에 참여할 수 있나요?\nKT 에이블스쿨은 풀타임으로 교육이 진행되며, 정해진 시간에 필수로 참여해야 합니다.',
                            '타사 교육과 병행 할 수 있나요?\nKT 에이블스쿨은 풀타임(09:00~18:00)으로 교육이 진행되므로 교육에 풀타임으로 참여할 수 있어야 교육 수강이 가능합니다.',
                            '핸드폰으로도 수강 할 수 있나요?\n실습 교육을 핸드폰으로 소화하기는 어렵습니다. 제공해드린 노트북을 사용해 주시기 바랍니다.',
                            '교육을 중도 포기할 경우 패널티가 있나요?\n교육 시작 후 7일(주말, 공휴일 포함) 이내에는 수강 철회가 가능합니다.\n다만, 이후에 수강을 철회하는 경우 국민내일배움카드 운영 기준에 따라 패널티가 부과되며,\n추후 국민내일배움카드를 활용하여 직업훈련 과정을 수강할 경우 자기부담액이 발생할 수 있습니다.\n1) 질병, 사고, 훈련기관 사정, 천재지변 등 불가피한 사유 없이 중도에 훈련 수강을 그만둔 경우\n(1회: 20만원, 2회: 50만원, 3회: 100만원 차감)\n2) 거짓이나 그 밖의 부정한 방법으로 출결체크하여 제적된 경우: 전액\n3) 지원대상에 해당하지 않거나 지원 제외 대상에 해당함에도 불구하고 거짓이나 그 밖의부정한 방법으로\n계좌를 발급받거나 지원을 받은 경우: 전액',
                            '교육과정 중 시험이 있나요?\nKT 에이블스쿨은 자기주도적인 학습을 장려하고 있으며, 교과목 종료 시 셀프테스트를 통한 역량 점검이 진행 됩니다.',
                            '교육 수강 중 졸업이 유예된 경우 어떻게 되나요?\n교육수강은 가능합니다. 다만 교육수료와 함께 이루어지는 채용 연계 시 참가한 기업에서 졸업자가 아니므로 채용하지 않을 수 있습니다. 해당 시점 채용공고에서 각 기업 별 내용을 직접 확인해야합니다.',
                            'KT 에이블스쿨은 무료인가요?\n네, KT 에이블스쿨은 ''K-Digital Training 교육''으로, 과정의 비용은 국민내일배움카드로 신청하여 지원받을 수 있습니다.',
                            '국민내일배움카드가 무엇인가요?\n국가에서 국민 스스로 직업능력개발훈련을 실시할 수 있도록 훈련비를 지원하는 제도로 2020년 이전 분리 운영되었던 실업자내일배움카드와 재직자내일배움카드가 통합되었습니다. 국민내일배움카드를 통해 1인당 300만 원에서 500만 원까지 지원받을 수 있습니다. * 지원대상 및 기준은 대상자별로 상이할 수 있으니 HRD-NET 사이트를 확인해주시기 바랍니다.',
                            '국민내일배움카드가 있어야만 지원이 가능한가요?\n네, 국민내일배움카드 보유자만 지원이 가능합니다. 현재 카드를 보유하고 있지 않더라도, K-Digital Training 교육 대상자로 확인되어 신규 발급하면 지원 가능합니다.',
                            '국민내일배움카드만 발급 받으면 KT 에이블스쿨 훈련과정을 들을 수 있는 건가요?\n아닙니다. 만 34세 이하의 정규 4년제 대학(전공무관) 졸업자 또는 졸업 예정자 중 국민내일배움카드 발급이 가능한자가 본 과정에 지원 할 수 있습니다. (계좌잔액 보유 有)',
                            '국민내일배움카드 계좌에 잔액이 없는데 어떡하나요?\nKT 에이블스쿨은 고용노동부와 KT가 공동으로 수행하는 청년 취업역량 강화 프로그램으로 국민내일배움카드 발급과 계좌내 잔액 보유가 필수입니다. 다만, 다른 훈련기관 등에서 K-Digital Training 교육을 수강한 이력이 있는 경우 훈련수강이 어려울 수 있으니 꼭 관할 고용센터에 확인 후 지원해 주세요.',
                            '훈련생이 추가 부담해야 하는 금액(훈련비용)이 있나요?\nKT 에이블스쿨은 고용노동부와 KT가 전액 훈련비용을 부담해서 운영합니다. 별도의 추가 훈련비용(자기부담금)이 없으니, 지원 요건에 충족한다면 즉시 지원해 주십시오.',
                            '국민내일배움카드는 몇 년간 사용할 수 있나요?\n계좌 발급일로부터 5년간 사용할 수 있습니다.',
                            '국민내일배움카드 발급 시 꼭 은행을 방문 해야하나요?\n국민내일배움카드 신청 시 "우편 발송"으로 선택하신 경우, 실물카드 수령까지 소요되는 시간이 신청자별로 상이할 수 있습니다. 다만, 훈련 운영의 효율을 위하여 "은행 방문"을 권장드립니다. 이미 "우편 발송"으로 선택하신 경우, 대한상공회의소(안내문자 발신처, 02-6050-3593)로 연락바랍니다.',
                            '국민내일배움카드 발급은 했는데, 수령까지 완료해야되나요?\n실물카드가 없어도 HRD-Net을 통해 국민내일배움카드 신청을 완료하였다면, 수강신청은 가능합니다. 카드 발급이 늦어져 카드 수령 전 HRD-Net 수강신청을 진행하는 경우, 수강신청 완료 후 대한상공회의소(안내문자 발신처, 02-6050-3593)로 연락바랍니다. 다만, 실물카드 수령이 확인되어야 KT 에이블스쿨 교육생 선발이 완료되오니, 최대한 발급 기한에 맞추어 발급해 주시기 바랍니다.',
                            '교육 진행 중 국민내일배움카드가 만료되는데 갱신해야되나요?\n추후 활용을 하실 수 있으니 갱신 발급을 권고 드립니다. 카드 발급 관련해서는 인터넷 또는 주소지 관할 고용센터를 통해 진행 하실 수 있으니 참고 부탁 드립니다.',
                            '타기관에서 K-Digital Training 교육과정을 수강했는데, 추가로 수강할 수 있나요?\nK-Digital Training 교육과정은 1회차에 한해서만 무료로 수강가능하며, 에이블스쿨은 자기부담금이 발생하지 않는 자를 지원 대상으로 하고 있어 추가 수강이 어렵습니다.',
                            'K-Digital Training 과정을 1번 수강하다가 저와 맞지않는 것 같아 수강을 포기 하였습니다. 또 K-Digital Training 교육과정을 들을 수 있나요?\n최초 수강한 K-Digital Training 교육과정을 수강철회기간에 취소하였다면 K-DT 과정을 들을 수 있습니다. 수강철회기간 이후에 중도 퇴교하였다면 수강이 어려울 수 있음을 참고 부탁 드립니다. 개인별 상황이 다를 수 있으므로 정확한 수강 가능 여부는 관할 고용센터에 문의하여 확인하시기 바랍니다. 관할 고용센터 정보는 고용노동부 대표번호 1350 또는 거주지 근처 지도검색, HRD-Net에서 확인 가능합니다.',
                            '훈련장려금이 무엇인가요?\n훈련장려금이란 긴 기간의 국비과정을 수강하는 분들에게 경제적 부담을 덜어주기 위해 교통비와 식비 명목으로 제공되는 지원금을 말합니다\nㅇ 훈련장려금 지급 조건\n1) 수강교육과정의 총 훈련시간이 140시간 이상\n2) 단위기간 출석률 80% (단위기간: 교육개강일로부터 매 1개월을 의미)\n3) 마지막 단위기간 시 수강평 작성\nㅇ 훈련장려금 지급 대상\n1) 실업자\n2) 주 15시간 미만 근로자\n3) 근로장려금 수급자\n4) 고용보험 임의가입 자영업자\n5) 국민취업제도 참여자\n(단, 총 훈련시간이 140시간 미만이라도 국민취업지원제도 1유형 참여자, 2유형 참여자 중 저소득층 참여자, 고용보험 임의가입 자영업자는 훈련장려금 수령 가능)',
                            '훈련장려금은 모두가 다 받을 수 있나요?\nKT 에이블스쿨 과정 수강생이라면 기본적으로 다 받을 수 있습니다. 다만, 실업급여 수급자 등은 수령이 불가 할 수 있으며, 출석 기준에 따라 수령 비용은 상이할 수 있습니다. 자세한 사항은 관할 고용센터에 문의 부탁 드립니다.',
                            '훈련장려금은 언제 나오나요?\n훈련장려금은 단위기간(훈련시작일로부터 1개월)의 출석률 등의 장려금 지급 기준을 총족하였을 때 수령 하실 수 있습니다. (단, 단위기간 출석률이 80% 이상인 경우만 입금되며, 실업급여 수급 중이거나 소득이 있는 경우 일부 부지급으로 개별 확인 필요) 훈련장려금은 통상 단위기간 종료일로부터 1개월 이내 지급됩니다. 다만, 고용센터 일정에 따라 권역별 지급일자가 상이할 수 있습니다.',
                            '훈련장려금은 얼마가 어떻게 지급되나요?\n훈련장려금은 월 최대 116,000원이 지급됩니다. 2023년의 경우 특별훈련수당이 200,000원 추가 지원되어, 월 최대 316,000원이 지급됩니다. 단위기간의 출석률이 80%이상인 경우 실업상태, 재직여부 및 소득 수준에 따라 달리 지급됩니다. 총 훈련시간이 140시간 이상인 국민내일배움카드 과정을 수강하는 경우 실업자, 주 15시간 미만 근로를 제공하는 고용보험 피보험자, 근로장려금 수급자, 고용보험 입의가입 자영업자는 지급받을 수 있습니다.',
                            '아르바이트를 하면 훈련장려금을 못받나요?\n훈련장려금은 실업자 지급이 원칙입니다. 다만, 주 15시간 미만으로 근로하며, 해당 근로사실과 동일하게 고용보험에 가입되어 있는 경우 예외적으로 훈련장려금이 지급됩니다. 더불어, 훈련 수강일정과 근로시간은 중복될 수 없습니다. 자세한 사항은 관할 고용센터에 문의 부탁 드립니다.',
                            '자영업자 고용보험 가입 시에는 훈련장려금 못받나요?\n자영업자 중 고용보험 임의가입자는 훈련장려금을 받으실 수 있습니다.',
                            '교내 장학금 수령 시 훈련장려금 못받나요?\n장학금이 소득으로 인식되는 경우 훈련장려금이 지급되지 않을 수 있습니다. 훈련장려금은 고용센터에서 지급요건을 최종 검토 후 지급처리 되므로, 먼저 학교 행정처에 장학금 소득 처리 여부 문의 후 거주지 관할 고용센터(HRD-Net 확인) 혹은 훈련 권역별 고용센터(아래 참고)로 연락하시어 훈련장려금 지급 여부에 대해 다시 한 번 문의하시기 바랍니다.\n※ 수도권: 성남고용센터\n충남/충북: 대전고용센터\n대구/경북: 대구고용센터\n전남/전북: 광주고용센터\n부산/경남: 부산고용센터\n고용노동부 대표번호 1350, 거주지 근처 지도검색, HRD-Net 메인 하단 지역별 기관검색에서 확인 가능',
                            '국민취업지원제도 6개월 경과 시 훈련장려금 못받나요?\n국민취업지원제도를 적용 받으시고 수당 수령 기간을 다 충족하셨다면 국민취업지원제도로 별도의 수당을 받으시기는 어렵습니다. 다만, 직업훈련 수강에 따른 별도의 훈련장려금은 출석률 등의 장려금 지급 기준을 충족할 경우 수령 하실 수 있습니다.',
                            '채용 연계 관련 어떤 지원이 있나요?\nKT 에이블스쿨은 미래인재육성 + 청년 IT/SW/DX 분야 일자리 창출을 위해서 KT 와 정부가 함께 하는 프로그램입니다. 에이블러 여러분들의 취업기회를 위해 Job Searching 세션, 취업컨설팅, 잡페어 등 여러 프로그램들을 운영하고 있습니다.',
                            'KT 채용 연계 시 중점적으로 보는 요소가 있나요?\nKT 그룹, AI 원팀 기업/스타트업 100 등 다양한 기업에서 채용 연계할 예정으로 기업마다 인재상 및 반영 요소가 다르기 때문에 중점 요소에 대해 안내드리기 어려운 점 양해 부탁드립니다. 채용에 대한 부분은 교육을 참여하시면 추후 관련 부서를 통해 안내드릴 예정입니다.',
                            '교육생에게는 어떤 사항들이 지원되나요?\n취업에 도움이 되는 다양한 프로젝트를 제공하고, 포트폴리오 작성을 도와드리며, 코딩챌린지, 해커톤 등 다양한 Competition 수상 기회를 제공합니다. 또한, AIVLE Job Fair를 거쳐 우수 수료생을 대상으로 KT와 KT그룹, AI원팀, 협력사 등 채용까지 연계해 드립니다.',
                            '참가자에게 노트북을 대여해주나요?\n네, 교육기간 동안 노트북을 무료로 지원하며, 교육에 필요한 필수 프로그램이 설치되어 있으므로, 제공한 노트북을 사용해 주시기 바랍니다.',
                            '교육기간 제공하는 노트북 사양이 궁금합니다.\n원활한 실습 환경을 위해 아래 명시된 사양 이상을 갖춘 노트북을 제공합니다. - CPU : I5 / RAM : 8GB / 여유저장공간 : 40GB',
                            '대면 교육 시 숙소가 제공 되나요?\n숙소는 제공하지 않습니다.',
                            '국비지원 IT 교육이 많은데 똑같은거 아닌가요? 뭐가 다른가요?\nKT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무에서 일하는 방식을 배울 수 있습니다. 또한, KT 현직 전문가가 전담하여 산업지식, AI/DX 기술, 창의성을 갖출 수 있도록 강의와 코칭을 제공하고, 강의 중 궁금한 사항은 전담 튜터가 1:1로 직접 코칭을 제공합니다. 특히, 우수 수료생을 대상으로 KT와 KT 그룹, AI원팀, 협력사 등 채용까지 연계해 드립니다.',
                            'AICE (기존 AIFB) 자격증이 있어야 유리한가요?\n네. AICE (기존 AIFB) Associate 자격 취득자는 선발 시 우대하고 있습니다.',
                            'AICE 시험관련 문의는 어디에 해야하나요?\nhelp@aice.study 으로 문의 하시기 바랍니다.',
                            '국민취업지원제도를 신청하고 싶습니다.\n국민취업지원제도 신청과 관련 사항은 거주지 관할 고용센터, 업무 대행 위탁기관으로 문의 부탁 드립니다. 주소지 관할 고용센터, 업무대항 위탁기관은 고용부 고객센터 대표번호 1350 을 통해서 확인하실 수 있습니다.',
                            '국민취업지원제도를 안해도 교육 참여되나요?\n국민취업제도를 신청하지 않아도 KT 에이블스쿨 교육에 참여하실 수 있습니다. 국민내일배움카드 발급 및 HRD-Net 수강신청은 필수 사항이니 정해진 기한 내 완료 부탁드립니다.',
                            '국민취업지원제도 관련 직업훈련탐색표가 필요합니다.\n직업훈련탐색표는 고용센터 또는 국민취업지원제도 민간위탁기관이 정한 양식에 따라 HRD-Net의 훈련과정을 참고하여 직접 작성해야하며, HRD-Net에서 확인이 어려운 경우 KT 에이블스쿨 훈련 정보는 대한상공회의소(02-6050-3593)로 요청하시면 됩니다.'
                            ]

In [6]:
ind = database.add_texts(input_list)
ind

['8168a27a-cce7-4eba-88e6-de52e31c233b',
 'b080a8b1-7ef1-47b1-b96e-6e34ed3572ba',
 '9055cc94-cea5-4c2d-91ef-7ea747b1833f',
 'a4c7030f-8165-4df9-a242-d3afbd814612',
 '15adf4b0-3bf0-4943-be5c-2b3c633aa47d',
 '86f3a85c-b2ad-4abb-be24-81ed57d3e8b4',
 '2c5ebcce-eb68-4826-851e-9ead97a076ae',
 '98140a88-d1f7-4090-a895-d92388a7932e',
 '597badcc-3d5c-4e4f-90f8-54499f489e55',
 'f455727d-ef80-44be-9d74-417f373ef16b',
 'd2fb7b9c-de5c-4bde-bb9f-caf4f2b39efe',
 '258fe356-62d0-4f99-860a-cffbc4542ac5',
 'f68c6cd3-3a46-4cd9-9ad9-4a6ecefb4e2e',
 '11a70db4-87a9-4a06-b28c-5b6c0494f424',
 '6c8b72e2-1d07-4b05-ac1f-929dd0f1cfcd',
 '411a01f8-cc66-4e10-b2f2-a52981c71d69',
 'bc71aad4-4b98-41fb-bc7b-498be95adeb4',
 '5abb3f10-411b-4942-8b9e-3fe161a16bd5',
 '3c61c018-76ca-4733-8da8-b8009181cdcb',
 '228aa6c8-4f38-45db-86e7-28bb099b1039',
 '9a27328e-8cde-49fc-bf1e-735e959211b9',
 '6ab08da5-692f-4b95-847e-5d53688aeec8',
 '3bfad885-ca1f-45cd-82ef-7642fa840a1d',
 '2c6b1527-ab54-4496-9873-28fa7f023f60',
 'dda9b6ea-ea9a-

In [7]:
database.get(ind)

{'ids': ['07d91ae5-2718-475b-985e-c0fb81299b39',
  '11a70db4-87a9-4a06-b28c-5b6c0494f424',
  '15adf4b0-3bf0-4943-be5c-2b3c633aa47d',
  '189b6c76-16e2-4825-ae3a-d2b41aa42741',
  '228aa6c8-4f38-45db-86e7-28bb099b1039',
  '22a9e5a2-0af4-4500-aef2-7a52f6caf18e',
  '258fe356-62d0-4f99-860a-cffbc4542ac5',
  '293fcf6e-94d9-4b3e-8e20-2877ba55166c',
  '2c5ebcce-eb68-4826-851e-9ead97a076ae',
  '2c6b1527-ab54-4496-9873-28fa7f023f60',
  '372af66e-720e-44b3-9c8b-72e437d593f6',
  '3b5234b4-c85c-4729-be32-011c5a3f93f6',
  '3bfad885-ca1f-45cd-82ef-7642fa840a1d',
  '3c61c018-76ca-4733-8da8-b8009181cdcb',
  '3d475bef-62e7-4e81-be4e-a8bfcfdfeca1',
  '411a01f8-cc66-4e10-b2f2-a52981c71d69',
  '49532186-8f87-494e-a6fe-6fbe01869914',
  '531ce35d-d3d1-46da-b907-2a5cf7eeb2f3',
  '597badcc-3d5c-4e4f-90f8-54499f489e55',
  '5abb3f10-411b-4942-8b9e-3fe161a16bd5',
  '63448fbd-3cfa-4764-bee8-9d6dace069b8',
  '65056805-fa20-4bb4-8d52-7a5122b850a3',
  '679b93e5-ece2-4b1a-b148-6f115e566fae',
  '6a6fa0ea-df0b-4c63-993f-

* 입력된 데이터 조회

In [17]:
# ① 연결
conn = sqlite3.connect(path)

# ② 작업 : select
df = pd.read_sql('SELECT * FROM test', conn)
display(df)

# ③ 연결 종료
conn.close()

# id 값이 자동으로 생성된다.

,구분,QA
0,모집/선발,최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제...
1,모집/선발,35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교...
2,모집/선발,미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함...
3,모집/선발,"직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시..."
4,모집/선발,아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 1...
...,...,...
63,기타,AICE (기존 AIFB) 자격증이 있어야 유리한가요?\n네. AICE (기존 AI...
64,기타,AICE 시험관련 문의는 어디에 해야하나요?\nhelp@aice.study 으로 문...
65,기타,국민취업지원제도를 신청하고 싶습니다.\n국민취업지원제도 신청과 관련 사항은 거주지 ...
66,기타,국민취업지원제도를 안해도 교육 참여되나요?\n국민취업제도를 신청하지 않아도 KT 에...


## **3.RAG+LLM모델**

* 모델 : ConversationalRetrievalChain
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정
    * memory 사용
* 요구사항
    * 질문 history 관리를 위한 이력 저장 DB 생성
        * DB 명 : db_chatlog
        * 테이블 명 : history
            * id INTEGER PRIMARY KEY : 이렇게 설정하면 자동증가 값으로 채워짐
            * datetime TEXT : 질문시점 yyyy-mm-dd hh:mi:ss
            * query TEXT : 질문
            * sim1 REAL : 첫번째 문서의 유사도 점수
            * sim2 REAL : 두번째 문서의 유사도 점수
            * sim3 REAL : 세번째 문서의 유사도 점수
            * answer TEXT : 답변
        * 유사도 점수는 similarity_search_with_score 메서드를 이용해서 저장해야 함
        * 질문과 답변이 진행될 때마다 history 테이블에 데이터 입력

* 관리용 DB, 테이블 생성

In [18]:
path = './db/db_aivle.db'
conn = sqlite3.connect(path)

In [19]:
# ① 연결
conn = sqlite3.connect(path)

# ② 작업 : select
data = pd.read_sql('SELECT * FROM test', conn)
display(data)

# ③ 연결 종료
conn.close()

# id 값이 자동으로 생성된다.

,구분,QA
0,모집/선발,최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제...
1,모집/선발,35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교...
2,모집/선발,미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함...
3,모집/선발,"직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시..."
4,모집/선발,아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 1...
...,...,...
63,기타,AICE (기존 AIFB) 자격증이 있어야 유리한가요?\n네. AICE (기존 AI...
64,기타,AICE 시험관련 문의는 어디에 해야하나요?\nhelp@aice.study 으로 문...
65,기타,국민취업지원제도를 신청하고 싶습니다.\n국민취업지원제도 신청과 관련 사항은 거주지 ...
66,기타,국민취업지원제도를 안해도 교육 참여되나요?\n국민취업제도를 신청하지 않아도 KT 에...


In [20]:
# 데이터프레임의 텍스트 열(시리즈)을 리스트로 변환
text_list = data['QA'].tolist()

# 리스트 내용을 각각 document로 변환
documents = [Document(page_content=text) for text in text_list]

print(documents)

[Document(page_content='최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.'), Document(page_content='35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교육입니다. 단, 모집시점에 35세라도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다. (예: 5기 지원 기준 1989년 1월 1일 이후 출생자)'), Document(page_content='미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함이 있는 근로인 경우,\n2) 고용보험에 미가입한 경우,\n3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우\n단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.'), Document(page_content='직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가능합니다. 향후 취업여부를 확인할 예정입니다.'), Document(page_content='아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 15시간/주 미만 근로인 경우에만 미취업자로 간주하여 지원이 가능합니다.\n고용보험에 가입되어 있지 않는 경우에는 특수형태근로자/고용보험미가입근로자로 분류되어 근로시간과 훈련시간이 중복되지 않는다면 훈련 수강은 가능하나, 훈련장려금이 지급되지 않습니다.\n다만, 특수형태근로자의 경우 해촉증명서, 퇴사 사실확인증명원(사업주, 근로자 직인/서명포함)을 제출하는 경우 훈련장려금 지급이 가능하므로,\n고용형태 관련 세부사항은 거주지 관할 고용센터(HRD-Net 확인) 또는 훈련 권역별 고용센터로 문의하시기

In [21]:
# Insert
database.add_documents(documents)

['a1f077e5-5a8d-446c-a6bb-da06d133cb47',
 '5744f4ea-06b2-42e9-ab43-7b3bd276d28b',
 '01260260-48a9-42d4-b10c-aaae31003f03',
 'a0a9cde0-5da2-4c3a-9062-bbe570ac12c4',
 'dc5e2d9c-5434-4687-9110-edf2be6ccdb4',
 '74d683a8-22b1-4cda-bd31-0c1e6c7b65ac',
 '6f208037-5fde-443c-98ac-8cfca81788dd',
 '0c620fd4-e75a-4a0b-987c-655669f16aad',
 '48e7950f-d665-4a9d-a48a-34be651a097f',
 'd2f50314-8ae8-4a6f-a650-b62bf19ae60c',
 'd9ce24c3-583e-4008-8dc3-ec7ae2c9f91c',
 'efff59ad-f746-4f24-8d6d-caa1d46b524b',
 '66bda63c-4a8b-4eec-bbbb-cebcc8d489e5',
 '26fb407c-4aa2-40ab-8057-3185e59d157f',
 '334d594e-e19b-4903-8ceb-f734d1a86890',
 '9007f6a3-32d5-49ab-a665-7853f95aed3f',
 'c0e904cd-328a-455d-957a-ede3994ecaf4',
 'd5f21273-b2c4-47c4-8a0c-931cb7b011c1',
 '0b10d92f-7480-4c94-9411-6606e52b55ef',
 '666f8f1e-ae79-44f6-b82a-3e88a9038abd',
 '34d47d2b-86cb-452b-be7a-f0ad8b3156d2',
 'e852d752-238d-4ad4-aad8-34380e95842f',
 'ce96aea5-f4e6-4d50-b5a0-047fd223b905',
 '4303826d-2cb7-486a-8e36-b1b2808f3560',
 '18e7a767-d829-

In [22]:
path = './db/db_chatlog.db'
conn = sqlite3.connect(path)

In [23]:
conn = sqlite3.connect(path)

# 커서 객체 생성
cursor = conn.cursor()

# test 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS log (
    id INTEGER PRIMARY KEY,
    datetime TEXT NOT NULL,
    query TEXT NOT NULL,
    sum1 REAL NOT NULL,
    sum2 REAL NOT NULL,
    sum3 REAL NOT NULL,
    answer TEXT NOT NULL)
''')

# 변경사항 커밋 (저장)
conn.commit()

# 연결 종료
conn.close()

In [24]:
path = './db/db_chatlog.db'
conn = sqlite3.connect(path)

In [31]:
# ① 연결
conn = sqlite3.connect(path)

# 문서 조회 : 유사도 점수도 함께 조회
query = input()   # 질문할 문장
k = 3                      # 유사도 상위 k 개 문서 가져오기.

# 현재 시간
dt = datetime.now()
dt = dt.strftime('%Y-%m-%d %H:%M:%S')

sum_arr = []
    
result = database.similarity_search_with_score(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(query)
print(result)
print('-'*50)
for doc in result:
    print(f"유사도 점수 : {round(doc[1], 5)}, 문서 내용: {doc[0].page_content}") # 문서 내용 표시
    sum_arr.append(round(doc[1], 5))
    doc = doc[0].page_content
    
    # ② 작업 : to_sql
data = pd.DataFrame({'datetime': [dt], 
                    'query': [query],
                    'sum1':[sum_arr[0]],
                    'sum2':[sum_arr[1]],
                    'sum3':[sum_arr[2]],
                    'answer':[doc]})
data.to_sql('history', conn, if_exists='append', index=False) # test 테이블이 있으면 insert, 없으면 생성

# ③ 연결 종료
conn.close()

몇시부터 시작하나요?
[(Document(page_content='출결 기준이 어떻게 될까요?\n당일 소정훈련시간의 50퍼센트 미만을 수강한 경우, 2. 지각, 조퇴 3회 누적 시, 결석 처리 됩니다.'), 0.368669331073761), (Document(page_content='미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함이 있는 근로인 경우,\n2) 고용보험에 미가입한 경우,\n3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우\n단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.'), 0.37038207054138184), (Document(page_content='미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함이 있는 근로인 경우,\n2) 고용보험에 미가입한 경우,\n3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우\n단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.'), 0.37038207054138184)]
--------------------------------------------------
유사도 점수 : 0.36867, 문서 내용: 출결 기준이 어떻게 될까요?
당일 소정훈련시간의 50퍼센트 미만을 수강한 경우, 2. 지각, 조퇴 3회 누적 시, 결석 처리 됩니다.
유사도 점수 : 0.37038, 문서 내용: 미취업자의 기준이 뭔가요?
미취업자의 기준은 아래와 같습니다.
1) 기간의 정함이 있는 근로인 경우,
2) 고용보험에 미가입한 경우,
3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우
단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.
유사도 점수 : 0.37038, 문서 내용: 미취업자의 기준이 뭔가요?
미취업자의 기준은 아래와 같습니다.
1) 기간의 정함이 있는

* 모델 선언

In [32]:
chat = ChatOpenAI(model="gpt-3.5-turbo")

* 모델 사용 및 이력 확인

In [35]:
k = 3
retriever = database.as_retriever(search_kwargs={"k": k})

qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True, )

result = qa(query)
result

{'query': '몇시부터 시작하나요?',
 'result': '저는 그 정보를 가지고 있지 않아요.',
 'source_documents': [Document(page_content='출결 기준이 어떻게 될까요?\n당일 소정훈련시간의 50퍼센트 미만을 수강한 경우, 2. 지각, 조퇴 3회 누적 시, 결석 처리 됩니다.'),
  Document(page_content='미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함이 있는 근로인 경우,\n2) 고용보험에 미가입한 경우,\n3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우\n단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.'),
  Document(page_content='미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함이 있는 근로인 경우,\n2) 고용보험에 미가입한 경우,\n3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우\n단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.')]}